In [1]:
import numpy as np 
import pandas as pd
import requests
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from datetime import datetime,timedelta
import datetime
import json
from bs4 import BeautifulSoup



In [2]:
current_date = datetime.date.today().strftime('%Y-%m-%d')
previous_date = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
url = f'https://finnhub.io/api/v1/company-news?symbol=TSLA&from={previous_date}&to={current_date}&token=cvinqmpr01qijvgji61gcvinqmpr01qijvgji620'
response = requests.get(url)

In [3]:
temp=response.json()
top_10_news = temp[:10]


In [4]:
dataset = {
    "Company": "Tesla",
    "Articles": [
        {
            "Title": article.get("headline", "No headline available"),
            "Summary": article.get("summary", "No summary available"),
            "url":article.get("url","No Url Available")
        }
        for article in top_10_news
    ]
}


In [5]:
var=json.dumps(dataset, indent=4)
print(var)

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc"
        },
        {
            "Title": "Tesla stock gains as it is seen as 'winner' from Trump's auto tariffs",
            "Summary": "Investing.com -- As the stock market weighs the fallout from President Trump\u2019s 25% tariffs on all foreign-made cars and light trucks, Tesla is a rare bright spot.",
            "url": "https://finnhub.io/api/news?id=8830879f2df6d8e5a53e219c28fed8fd8f7263eb4debfa3773b42dd816171f82"
        },
        {
            "Title": "Why Tesla (TSLA) Stock Is Moving Today",
       

In [ ]:
if isinstance(var, str):  
    var = json.loads(var)
    
def extract_webpage_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')           
        main_content = soup.find('article') or soup.find('main') or soup.find('div', class_='content')   
        if main_content:
            content_text = main_content.get_text(strip=True)
        else:
            content_text = 'Could not extract main content'
        return {
            'content': content_text
        }
    
    except requests.RequestException as e:
        return f"Error fetching the URL: {e}"


for article in var["Articles"]:
    finnhub_url = article.get("url", "")
    print(f"Original Finnhub URL: {finnhub_url}")
    article["Content"] = extract_webpage_content(finnhub_url) if finnhub_url else "No URL provided"
    print(json.dumps(var, indent=4))





In [7]:
flag1=json.dumps(var, indent=4)


In [11]:
print(flag1)

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc",
            "Content": {
                "content": "Unlock stock picks and a broker-level newsfeed that powers Wall Street.Upgrade NowElon Musk says Trump's auto tariffs won't be a problem for TeslaNew tariffs on trade partners? Not a problem. Get rid of the EV tax credit? Sure thing. Trump's moves appear to hurt Tesla's competitors more than Tesla.Pras Subramanian\u00b7Senior ReporterUpdatedThu, Mar 27, 2025, 11:51 PM4 min readIn This Article:TSLA+1.77%RIVN+7.68%F-2.67%GM-6.34%Automaker stocks here and abroad

In [ ]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
import re

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=1024,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def get_sentiment_from_groq(text):
    prompt = f""" 
    Analyze the sentiment of the following news article excerpt. Classify the sentiment as either Positive, Negative, or Neutral. Respond *ONLY* with the single word "Positive", "Negative", or "Neutral", without any additional explanation. Your response MUST be one of these three options. 

    News Article Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"
    
def extract_topics_from_groq(text):
    prompt = f""" 
    Extract the top 3-5 key topics from the following news article excerpt. 
    Respond ONLY with a comma-separated list of topic names. 
    Do NOT include any descriptions, explanations, or additional text.
    Focus on the most significant and specific topics discussed in the text.
    Provide ONLY topic names as simple strings.

    News Article Excerpt: 
    {text} 

    Topics (comma-separated):
    """

    response = agent.run(prompt)

    def clean_topic(topic):
        return re.sub(r'^["\']|["\']$', '', topic.strip())

    topics = [clean_topic(topic) for topic in response.split(',') if clean_topic(topic)]
    
    return topics[:5]


agent = Agent(model_name="llama3-8b-8192")

if isinstance(flag1, str):
    flag1 = json.loads(flag1)

for article in flag1["Articles"]:
    content_text = article["Content"]["content"]
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    topics= extract_topics_from_groq(content_text)
    article["Topics"] = topics

print(json.dumps(flag1, indent=4))

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc",
            "Content": {
                "content": "Unlock stock picks and a broker-level newsfeed that powers Wall Street.Upgrade NowElon Musk says Trump's auto tariffs won't be a problem for TeslaNew tariffs on trade partners? Not a problem. Get rid of the EV tax credit? Sure thing. Trump's moves appear to hurt Tesla's competitors more than Tesla.Pras Subramanian\u00b7Senior ReporterUpdatedThu, Mar 27, 2025, 11:51 PM4 min readIn This Article:TSLA+1.77%RIVN+7.68%F-2.67%GM-6.34%Automaker stocks here and abroad

In [25]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
from collections import Counter

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=1024,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def get_sentiment_from_groq(text):
    prompt = f""" 
    Analyze the sentiment of the following news article excerpt. Classify the sentiment as either Positive, Negative, or Neutral. Respond *ONLY* with the single word "Positive", "Negative", or "Neutral", without any additional explanation. Your response MUST be one of these three options. 

    News Article Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"

def extract_topics_from_groq(text):
    prompt = f""" 
    Extract the top 3-5 key topics from the following news article excerpt. 
    Respond ONLY with a comma-separated list of topic names. 
    Do NOT include any descriptions, explanations, or additional text.
    Focus on the most significant and specific topics discussed in the text.
    Provide ONLY topic names as simple strings.

    News Article Excerpt: 
    {text} 

    Topics (comma-separated):
    """

    response = agent.run(prompt)

    # Clean and process the response
    def clean_topic(topic):
        # Remove any quotes, extra whitespace, and strip
        return re.sub(r'^["\']|["\']$', '', topic.strip())

    # Split by comma, clean each topic, remove empty strings
    topics = [clean_topic(topic) for topic in response.split(',') if clean_topic(topic)]
    
    # Limit to 5 topics
    return topics[:5]

def generate_comparative_analysis(articles):
    # Sentiment Distribution
    sentiment_distribution = Counter(article['Sentiment'] for article in articles)
    
    # Topic Overlap
    all_topics = [topic for article in articles for topic in article['Topics']]
    unique_topics = [set(article['Topics']) for article in articles]
    common_topics = set.intersection(*unique_topics) if unique_topics else []
    
    # Unique Topics per Article
    unique_topics_per_article = [
        set(article['Topics']) - set.union(*(unique_topics[:i] + unique_topics[i+1:]))
        for i, article in enumerate(articles)
    ]
    
    # Coverage Differences Prompt
    coverage_prompt = f"""
    Compare and analyze the following news articles about Tesla:
    
    Articles:
    {json.dumps(articles, indent=2)}
    
    Provide a detailed comparative analysis focusing on:
    1. Key differences in narrative and tone
    2. Contrasting perspectives
    3. Potential impact on investor perception
    
    Respond with a concise analysis of coverage differences and their potential implications.
    """
    
    # Get coverage differences analysis from Groq
    coverage_differences = agent.run(coverage_prompt)
    
    # Final Sentiment Analysis Prompt
    final_sentiment_prompt = f"""
    Based on the following articles about Tesla, provide a brief overall sentiment analysis:
    
    Articles:
    {json.dumps(articles, indent=2)}
    
    Consider:
    - Overall sentiment trend
    - Key positive and negative factors
    - Potential implications for Tesla
    
    Provide a concise, forward-looking summary.
    """
    
    # Get final sentiment analysis from Groq
    final_sentiment_analysis = agent.run(final_sentiment_prompt)
    
    # Construct Comparative Analysis
    comparative_analysis = {
        "Comparative Sentiment Score": {
            "Sentiment Distribution": dict(sentiment_distribution),
            "Coverage Differences": [
                {
                    "Comparison": coverage_differences.split('\n')[0],
                    "Impact": coverage_differences.split('\n')[1] if len(coverage_differences.split('\n')) > 1 else ""
                }
            ],
            "Topic Overlap": {
                "Common Topics": list(common_topics),
                "Unique Topics in Article 1": list(unique_topics_per_article[0]) if unique_topics_per_article else [],
                "Unique Topics in Article 2": list(unique_topics_per_article[1]) if len(unique_topics_per_article) > 1 else []
            }
        },
        "Final Sentiment Analysis": final_sentiment_analysis
    }
    
    return comparative_analysis

# Initialize the agent with the specific model
agent = Agent(model_name="llama3-8b-8192")

# Assuming flag1 is a JSON string or dictionary loaded elsewhere
if isinstance(flag1, str):
    flag1 = json.loads(flag1)

# Process each article
for article in flag1["Articles"]:
    content_text = article["Content"]["content"]
    
    # Get sentiment
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    
    # Extract topics
    topics = extract_topics_from_groq(content_text)
    article["Topics"] = topics

# Generate Comparative Analysis
flag1["Comparative Analysis"] = generate_comparative_analysis(flag1["Articles"])

print(json.dumps(flag1, indent=4))

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama3-8b-8192` in organization `org_01jp7ne9g5ehf96mk2vxq3nafq` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 8488, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama3-8b-8192` in organization `org_01jp7ne9g5ehf96mk2vxq3nafq` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 8492, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [27]:
import os
import json
import re
from dotenv import load_dotenv
from groq import Groq
from collections import Counter

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=512,  # Reduced max tokens
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def truncate_text(text, max_tokens=500):
    """
    Truncate text to approximately max_tokens words
    Helps reduce overall token count
    """
    words = text.split()
    return ' '.join(words[:max_tokens])

def get_sentiment_from_groq(text):
    # Truncate text to reduce token usage
    text = truncate_text(text)
    
    prompt = f""" 
    Analyze the sentiment of this news excerpt. 
    Respond with ONLY "Positive", "Negative", or "Neutral".

    Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"

def extract_topics_from_groq(text):
    # Truncate text to reduce token usage
    text = truncate_text(text)
    
    prompt = f""" 
    List 3-5 key topics from this news excerpt. 
    Respond with comma-separated topic names only.

    Excerpt: 
    {text} 

    Topics:
    """

    response = agent.run(prompt)

    # Clean and process the response
    def clean_topic(topic):
        return re.sub(r'^["\']|["\']$', '', topic.strip())

    # Split by comma, clean each topic, remove empty strings
    topics = [clean_topic(topic) for topic in response.split(',') if clean_topic(topic)]
    
    return topics[:5]

def generate_comparative_analysis(articles):
    # Sentiment Distribution
    sentiment_distribution = Counter(article['Sentiment'] for article in articles)
    
    # Topic processing
    all_topics = [topic for article in articles for topic in article['Topics']]
    unique_topics = [set(article['Topics']) for article in articles]
    common_topics = set.intersection(*unique_topics) if unique_topics else []
    
    # Unique Topics per Article
    unique_topics_per_article = [
        set(article['Topics']) - set.union(*(unique_topics[:i] + unique_topics[i+1:]))
        for i, article in enumerate(articles)
    ]
    
    # Simplified comparison prompt
    comparison_prompt = f"""
    Compare these news articles about Tesla:
    - Article 1 Topics: {articles[0]['Topics']}
    - Article 2 Topics: {articles[1]['Topics']}
    - Article 1 Sentiment: {articles[0]['Sentiment']}
    - Article 2 Sentiment: {articles[1]['Sentiment']}

    Provide a brief analysis of coverage differences and their implications.
    """
    
    # Get coverage differences 
    coverage_differences = agent.run(comparison_prompt)
    
    # Final sentiment analysis prompt
    final_sentiment_prompt = f"""
    Analyze Tesla news coverage:
    - Sentiment Distribution: {dict(sentiment_distribution)}
    - Common Topics: {list(common_topics)}

    Provide a concise overall sentiment summary.
    """
    
    # Get final sentiment analysis
    final_sentiment_analysis = agent.run(final_sentiment_prompt)
    
    # Construct Comparative Analysis
    comparative_analysis = {
        "Comparative Sentiment Score": {
            "Sentiment Distribution": dict(sentiment_distribution),
            "Coverage Differences": [
                {
                    "Comparison": coverage_differences.split('\n')[0],
                    "Impact": coverage_differences.split('\n')[1] if len(coverage_differences.split('\n')) > 1 else ""
                }
            ],
            "Topic Overlap": {
                "Common Topics": list(common_topics),
                "Unique Topics in Article 1": list(unique_topics_per_article[0]) if unique_topics_per_article else [],
                "Unique Topics in Article 2": list(unique_topics_per_article[1]) if len(unique_topics_per_article) > 1 else []
            }
        },
        "Final Sentiment Analysis": final_sentiment_analysis
    }
    
    return comparative_analysis

# Initialize the agent with the specific model
agent = Agent(model_name="llama3-8b-8192")

# Assuming flag1 is a JSON string or dictionary loaded elsewhere
if isinstance(flag1, str):
    flag1 = json.loads(flag1)

# Process each article
for article in flag1["Articles"]:
    content_text = article["Content"]["content"]
    
    # Get sentiment
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    
    # Extract topics
    topics = extract_topics_from_groq(content_text)
    article["Topics"] = topics

# Generate Comparative Analysis
flag1["Comparative Analysis"] = generate_comparative_analysis(flag1["Articles"])

print(json.dumps(flag1, indent=4))

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc",
            "Content": {
                "content": "Unlock stock picks and a broker-level newsfeed that powers Wall Street.Upgrade NowElon Musk says Trump's auto tariffs won't be a problem for TeslaNew tariffs on trade partners? Not a problem. Get rid of the EV tax credit? Sure thing. Trump's moves appear to hurt Tesla's competitors more than Tesla.Pras Subramanian\u00b7Senior ReporterUpdatedThu, Mar 27, 2025, 11:51 PM4 min readIn This Article:TSLA+1.77%RIVN+7.68%F-2.67%GM-6.34%Automaker stocks here and abroad

In [28]:
import os
import json
import re
from dotenv import load_dotenv
from groq import Groq
from collections import Counter

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=512,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def truncate_text(text, max_tokens=500):
    words = text.split()
    return ' '.join(words[:max_tokens])

def get_sentiment_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Analyze the sentiment of this news excerpt. 
    Respond with ONLY "Positive", "Negative", or "Neutral".

    Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"

def extract_topics_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Extract 3-5 precise, distinct topics from this news excerpt. 
    Respond ONLY with comma-separated topic names. 
    Avoid generic terms. Be specific and meaningful.

    Excerpt: 
    {text} 

    Specific Topics:
    """

    response = agent.run(prompt)

    def clean_topic(topic):
        # Remove any lingering instructions or extra text
        topic = re.sub(r'^(Here are the topics:|Topics:|3-5 key topics:)', '', topic, flags=re.IGNORECASE)
        topic = re.sub(r'^["\']|["\']$', '', topic.strip())
        return topic if topic else None

    # Split, clean, and filter topics
    topics = [clean_topic(topic) for topic in response.split(',')]
    topics = [topic for topic in topics if topic]
    
    return topics[:5]

def generate_comparative_analysis(articles):
    # Determine topic contexts for each article
    article_contexts = []
    for article in articles:
        context_prompt = f"""
        Provide a concise 1-2 sentence summary of the primary focus and key narrative of this article:
        
        Article Content: {truncate_text(article['Content']['content'])}
        
        Focus and Narrative:
        """
        context = agent.run(context_prompt)
        article_contexts.append(context)

    # Comparative analysis prompt
    comparison_prompt = f"""
    Compare these two news articles about Tesla:
    
    Article 1 Context: {article_contexts[0]}
    Article 2 Context: {article_contexts[1]}
    
    Article 1 Topics: {articles[0]['Topics']}
    Article 2 Topics: {articles[1]['Topics']}
    
    Provide:
    1. A comparison of narrative differences
    2. Potential impact on investor perception
    3. Contrasting perspectives
    
    Response format:
    Comparison: [Narrative comparison]
    Impact: [Investor/market impact]
    """

    # Get detailed comparative analysis
    comparison_response = agent.run(comparison_prompt)
    
    # Parse comparison response
    comparison_lines = comparison_response.split('\n')
    comparison = next((line.replace('Comparison: ', '').strip() for line in comparison_lines if line.startswith('Comparison:')), '')
    impact = next((line.replace('Impact: ', '').strip() for line in comparison_lines if line.startswith('Impact:')), '')

    # Sentiment Distribution
    sentiment_distribution = Counter(article['Sentiment'] for article in articles)
    
    # Topic processing
    all_topics = [topic for article in articles for topic in article['Topics']]
    unique_topics = [set(article['Topics']) for article in articles]
    common_topics = set.intersection(*unique_topics) if unique_topics else []
    
    # Unique Topics per Article
    unique_topics_per_article = [
        set(article['Topics']) - set.union(*(unique_topics[:i] + unique_topics[i+1:]))
        for i, article in enumerate(articles)
    ]

    # Final sentiment analysis
    final_sentiment_prompt = f"""
    Analyze Tesla news coverage:
    - Article Sentiments: {[article['Sentiment'] for article in articles]}
    - Common Themes: {list(common_topics)}

    Provide a concise overall sentiment summary focusing on potential market implications.
    """
    
    final_sentiment_analysis = agent.run(final_sentiment_prompt)
    
    # Construct Comparative Analysis
    comparative_analysis = {
        "Comparative Sentiment Score": {
            "Sentiment Distribution": dict(sentiment_distribution),
            "Coverage Differences": [
                {
                    "Comparison": comparison,
                    "Impact": impact
                }
            ],
            "Topic Overlap": {
                "Common Topics": list(common_topics),
                "Unique Topics in Article 1": list(unique_topics_per_article[0]) if unique_topics_per_article else [],
                "Unique Topics in Article 2": list(unique_topics_per_article[1]) if len(unique_topics_per_article) > 1 else []
            }
        },
        "Final Sentiment Analysis": final_sentiment_analysis
    }
    
    return comparative_analysis

# Initialize the agent with the specific model
agent = Agent(model_name="llama3-8b-8192")

# Assuming flag1 is a JSON string or dictionary loaded elsewhere
if isinstance(flag1, str):
    flag1 = json.loads(flag1)

# Process each article
for article in flag1["Articles"]:
    content_text = article["Content"]["content"]
    
    # Get sentiment
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    
    # Extract topics
    topics = extract_topics_from_groq(content_text)
    article["Topics"] = topics

# Generate Comparative Analysis
flag1["Comparative Analysis"] = generate_comparative_analysis(flag1["Articles"])

print(json.dumps(flag1, indent=4))

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc",
            "Content": {
                "content": "Unlock stock picks and a broker-level newsfeed that powers Wall Street.Upgrade NowElon Musk says Trump's auto tariffs won't be a problem for TeslaNew tariffs on trade partners? Not a problem. Get rid of the EV tax credit? Sure thing. Trump's moves appear to hurt Tesla's competitors more than Tesla.Pras Subramanian\u00b7Senior ReporterUpdatedThu, Mar 27, 2025, 11:51 PM4 min readIn This Article:TSLA+1.77%RIVN+7.68%F-2.67%GM-6.34%Automaker stocks here and abroad

In [ ]:
import os
import json
import re
from dotenv import load_dotenv
from groq import Groq
from collections import Counter

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=512,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def truncate_text(text, max_tokens=500):
    words = text.split()
    return ' '.join(words[:max_tokens])

def get_sentiment_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Analyze the sentiment of this news excerpt. 
    Respond with ONLY "Positive", "Negative", or "Neutral".

    Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"

def extract_topics_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Extract 3-5 precise, distinct topics from this news excerpt. 
    Respond ONLY with comma-separated topic names. 
    Avoid generic terms. Be specific and meaningful.

    Excerpt: 
    {text} 

    Specific Topics:
    """

    response = agent.run(prompt)

    def clean_topic(topic):
        # Remove any lingering instructions or extra text
        topic = re.sub(r'^(Here are the topics:|Topics:|3-5 key topics:)', '', topic, flags=re.IGNORECASE)
        topic = re.sub(r'^["\']|["\']$', '', topic.strip())
        return topic if topic else None

    # Split, clean, and filter topics
    topics = [clean_topic(topic) for topic in response.split(',')]
    topics = [topic for topic in topics if topic]
    
    return topics[:5]

def generate_coverage_differences(articles):
    # Ensure we have at least two articles
    if len(articles) < 2:
        return [{"Comparison": "Insufficient articles for comparative analysis.", "Impact": "No comparative insights available."}]
    
    # Generate a detailed prompt for coverage differences
    coverage_prompt = f"""
    Analyze the following two articles about Tesla:

    Article 1:
    - Title: {articles[0].get('Title', 'Untitled')}
    - Topics: {articles[0].get('Topics', [])}
    - Sentiment: {articles[0].get('Sentiment', 'Unspecified')}
    - Summary: {articles[0].get('Summary', 'No summary available')}

    Article 2:
    - Title: {articles[1].get('Title', 'Untitled')}
    - Topics: {articles[1].get('Topics', [])}
    - Sentiment: {articles[1].get('Sentiment', 'Unspecified')}
    - Summary: {articles[1].get('Summary', 'No summary available')}

    Provide a detailed comparison that focuses on:
    1. Narrative differences between the two articles
    2. Contrasting perspectives
    3. Potential market or investor implications

    Format your response as:
    Comparison: [Specific narrative differences]
    Impact: [Market or investor implications]
    """

    try:
        coverage_response = agent.run(coverage_prompt)
        
        # Extract Comparison and Impact
        comparison_match = re.search(r'Comparison:\s*(.+?)(?=Impact:|$)', coverage_response, re.DOTALL)
        impact_match = re.search(r'Impact:\s*(.+)$', coverage_response, re.DOTALL)
        
        comparison = comparison_match.group(1).strip() if comparison_match else (
            "Articles present different perspectives on Tesla's recent developments."
        )
        impact = impact_match.group(1).strip() if impact_match else (
            "Varying coverage may create mixed signals for investors and market perception."
        )

        return [{
            "Comparison": comparison,
            "Impact": impact
        }]
    
    except Exception as e:
        # Fallback if analysis generation fails
        return [{
            "Comparison": "Diverse coverage highlighting different aspects of Tesla's business.",
            "Impact": "Multiple perspectives provide a nuanced view of the company's current situation."
        }]

# Modify the main script to use this function
def generate_comparative_analysis(articles):
    # Existing code for sentiment distribution and topic processing
    sentiment_distribution = Counter(article['Sentiment'] for article in articles)
    
    all_topics = [topic for article in articles for topic in article['Topics']]
    unique_topics = [set(article['Topics']) for article in articles]
    common_topics = set.intersection(*unique_topics) if unique_topics else []
    
    # Unique Topics per Article
    unique_topics_per_article = [
        set(article['Topics']) - set.union(*(unique_topics[:i] + unique_topics[i+1:]))
        for i, article in enumerate(articles)
    ]

    # Generate coverage differences
    coverage_differences = generate_coverage_differences(articles)
    
    final_sentiment_prompt = f"""
    Analyze Tesla news coverage:
    - Article Sentiments: {[article['Sentiment'] for article in articles]}
    - Common Themes: {list(common_topics)}

    Provide a concise overall sentiment summary focusing on potential market implications.
    """
    
    final_sentiment_analysis = agent.run(final_sentiment_prompt)

    # Construct Comparative Analysis
    comparative_analysis = {
        "Comparative Sentiment Score": {
            "Sentiment Distribution": dict(sentiment_distribution),
            "Coverage Differences": coverage_differences,
            "Topic Overlap": {
                "Common Topics": list(common_topics),
                "Unique Topics in Article 1": list(unique_topics_per_article[0]) if unique_topics_per_article else [],
                "Unique Topics in Article 2": list(unique_topics_per_article[1]) if len(unique_topics_per_article) > 1 else []
            }
        },
        "Final Sentiment Analysis":final_sentiment_analysis
    }
    
    return comparative_analysis

# Initialize the agent with the specific model
agent = Agent(model_name="llama3-8b-8192")

# Assuming flag1 is a JSON string or dictionary loaded elsewhere
if isinstance(flag1, str):
    flag1 = json.loads(flag1)

# Process each article
for article in flag1["Articles"]:
    content_text = article["Content"]["content"]
    
    # Get sentiment
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    
    # Extract topics
    topics = extract_topics_from_groq(content_text)
    article["Topics"] = topics

flag1["Comparative Analysis"] = generate_comparative_analysis(flag1["Articles"])

print(json.dumps(flag1, indent=4))

{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Why Tesla and CEO Elon Musk don't see Trump's auto tariffs as a problem",
            "Summary": "Automaker stocks here and abroad are getting smoked following President Trump's big move to impose 25% tariffs on foreign autos and certain auto parts. But one automaker is up \u2014 Tesla.",
            "url": "https://finnhub.io/api/news?id=24315bd241843e8bb28cb3bb6d839a3cd7b869930666cea1a80cc46f259516cc",
            "Content": {
                "content": "Unlock stock picks and a broker-level newsfeed that powers Wall Street.Upgrade NowElon Musk says Trump's auto tariffs won't be a problem for TeslaNew tariffs on trade partners? Not a problem. Get rid of the EV tax credit? Sure thing. Trump's moves appear to hurt Tesla's competitors more than Tesla.Pras Subramanian\u00b7Senior ReporterUpdatedThu, Mar 27, 2025, 11:51 PM4 min readIn This Article:TSLA+1.77%RIVN+7.68%F-2.67%GM-6.34%Automaker stocks here and abroad